## <center>DataLab Cup 1: Text Feature Engineering</center>

+ Team Name: 陳瑜旋轉陳玟旋轉陳瑜旋
+ Team Member: 111501528 謝旻峰 111501538 劉杰閎 111062588 陳玟璇 111062697 吳律穎

#### - To import the libraries

In [1]:
import pandas as pd
import numpy as np
import os

from bs4 import BeautifulSoup
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import nltk
import re

from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score

import joblib
import threading
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import itertools

from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier


import warnings
warnings.filterwarnings("ignore")

#### 1. To load the datasets
讀取訓練及測試資料集。

In [2]:
train_data = pd.read_csv('../dataset/train.csv')
test_data  = pd.read_csv('../dataset/test.csv')


#### 2. Preprocessing: To extract the features from the dataset

將一些我們想要用到的feature從dataset中提取出來。以下為提取的特徵:

BeautifulSoup 為 Python 的函式庫，可以從 HTML、XML 檔案中分析資料，從 Raw Data (News Content) 提取不同的 Tag 開頭來分析本次 Competition 中所使用到的 data。
vaderSentiment 也是 Python 的函式庫，可以對內容做情感分析，來當作一個特徵。

- title: 文章標題
- time(year/month/day/hour/minute/second): 文章所發表的日期時間 (包含年月日時分秒)
- number of images (num_img): 文章中圖片數量
- number of videos (num_video):  文章中影片數量
- author name: 文章作者的名字
- topic: 文章所屬的章節主題
- channel: 文章所屬的頻道
- length of content: 文章長度
- sentiment_neg: 代表文本或標題的負面情感得分。
- sentiment_neu: 代表文本或標題的中性情感得分。
- sentiment_pos: 代表文本或標題的積極情感得分。
- sentiment_compound: 代表文本或標題的綜合情感得分。

透過 BeautifulSoup 的 preprocessing 後，總共整理出 19 項資訊。

--- 

* beautiful soup
    - conda install -c conda-forge beautifulsoup4
    
<br>

* vadersentiment
    - conda install -c conda-forge vadersentiment

---

In [3]:
# to get the attribute of the 'title', 'year/month/date/day/hour/minute/second/is_weekend', 'num_img', 'num_video', 'author name', 'topic', 'channel', 'content length', 'title_sentiment'

def preprocessor(text):
    soup = BeautifulSoup(text, 'html.parser')

    # 1. to find the 'title' (body > h1)
    title = soup.find('h1').string.strip().lower()

    # 2. to find time(body > div > span > time)
    
    date_string = soup.find('time')
    try:
        date_string = date_string['datetime']
    except:
        date_string = 'wed, 10 oct 2014 15:00:43 +0000'
        
    date_string = date_string.strip().lower()
    datetimes = datetime.strptime(date_string, '%a, %d %b %Y %H:%M:%S %z')
    
    year = datetimes.year
    month = datetimes.month
    date = datetimes.day
    day = pd.Timestamp(str(year)+'-'+str(month)+'-'+str(date)).dayofweek+1
    is_weekend = 1 if (day==6 or day==7) else 0
    hour = datetimes.hour
    minute = datetimes.minute
    second = datetimes.second

    # 3. to find the number of images
    num_img  = len(soup.find_all('img'))
    

    # 4. to find the number of videos
    num_video = len(soup.find_all('iframe'))
    
    # 5. to find the author name
    article_info = soup.find('div', class_='article-info')
    author = article_info.find('span', class_='author_name') or article_info.find('span', class_='byline basic')

    if (author != None):
        if (author.find('a') != None):
            author = author.find('a')
            author_name = author.get_text().lower()
        else :
            author_name = author.get_text().lower()
    else :
        author_name = 'not found'

    # 6. to find the article topic
    footer = soup.find('footer', class_='article-topics')
    topic = footer.get_text().split(': ')[1]
    
    # 7. to find the channel
    channel = soup.find('article')['data-channel'].strip().lower()
    
    
    # 8. to find the content length
    content = soup.body.find('section', class_='article-content').get_text()
    len_content = len(content)

    # print('topic = ', topic, type(topic))
    
    # 9. to find the sentiment of title
    analyzer = SentimentIntensityAnalyzer()
    title_sentiment = analyzer.polarity_scores(topic)
    sentiment_neg = title_sentiment['neg']
    sentiment_neu = title_sentiment['neu']
    sentiment_pos = title_sentiment['pos']
    sentiment_compound = title_sentiment['compound']

    return title, author_name, channel, topic, year, month, date, day, is_weekend, hour, minute, second, num_img, num_video, len_content, sentiment_neg, sentiment_neu, sentiment_pos, sentiment_compound

In [4]:
feature_train_list = []
feature_test_list = []

for content in (train_data['Page content']):
    feature_train_list.append(preprocessor(content))
for content in (test_data['Page content']):
    feature_test_list.append(preprocessor(content))

df_train_x = pd.DataFrame(
        feature_train_list, 
                columns=['title', 'author_name', 'channel', 'topic', 'year', 'month', 'date', 'day', \
                 'is_weekend', 'hour', 'minute', 'second', 'num_img', 'num_video', 'len_content', \
                 'sentiment_neg', 'sentiment_neu', 'sentiment_pos', 'sentiment_compound']).values
df_test_x = pd.DataFrame(
        feature_test_list, 
        columns=['title', 'author_name', 'channel', 'topic', 'year', 'month', 'date', 'day', \
                 'is_weekend', 'hour', 'minute', 'second', 'num_img', 'num_video', 'len_content', \
                 'sentiment_neg', 'sentiment_neu', 'sentiment_pos', 'sentiment_compound']).values

df_train_y = train_data['Popularity'].values
df_train_y[df_train_y == -1] = 0



### 3-1. Preprocessing - Word Stemming
透過 Column_Transformer 處理指定的 feature。針對文字資訊 (Author, Topic) 會先做 Tokenization 提取出每一個單字，接下來使用 WordNetLemmatizer 對單字做精確度較高的字根還原，最後用 TfidfVectorizer 計算每個range的 TF-IDF 成績，作為文字資訊轉換過後的 features。


In [5]:
nltk.download('wordnet')
nltk.download('omw-1.4')

def word_stemming(text):
    if type(text) == np.ndarray:
        text = text[0]
    lm = WordNetLemmatizer()
    words = re.split('\s', text.strip())
    lemmatized_words = [lm.lemmatize(word) for word in words]
    return lemmatized_words

[nltk_data] Downloading package wordnet to
[nltk_data]     /users/student/mr111//mfhsieh22/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /users/student/mr111//mfhsieh22/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


#### 3-5 Create TF-IDF feature representation ([ref](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html))

In [6]:
xgb_transformer = ColumnTransformer(
    [
        ('drop process', 'drop', [0, 2, 10, 11, 12]),
        ('author name process', TfidfVectorizer(tokenizer=word_stemming, token_pattern=None, ngram_range=(1,2), lowercase=False), [1]),
        ('topic name process' , TfidfVectorizer(tokenizer=word_stemming, token_pattern=None, ngram_range=(1,2), lowercase=False), [3]),
    ],
    remainder='passthrough', # do not touch the remaining data
    n_jobs=-1
)
lgb_transformer = ColumnTransformer(
    [
        ('drop process', 'drop', [0, 2, 10, 11, 12, 15, 16, 17, 18]),
        ('author name process', TfidfVectorizer(tokenizer=word_stemming, token_pattern=None, ngram_range=(1,2), lowercase=False), [1]),
        ('topic name process' , TfidfVectorizer(tokenizer=word_stemming, token_pattern=None, ngram_range=(1,2), lowercase=False), [3])
    ],
    remainder='passthrough', # do not touch the remaining data
    n_jobs=-1
)
cat_transformer = ColumnTransformer(
    [
        ('drop process', 'drop', [0, 2, 10, 11, 12]),
        ('author name process', TfidfVectorizer(tokenizer=word_stemming, token_pattern=None, ngram_range=(1,2), lowercase=False), [1]),
        ('topic name process' , TfidfVectorizer(tokenizer=word_stemming, token_pattern=None, ngram_range=(1,1), lowercase=False), [3])
    ],
    remainder='passthrough', # do not touch the remaining data
    n_jobs=-1
)
ada_transformer = ColumnTransformer(
    [
        ('drop process', 'drop', [0, 2, 10, 11, 12]),
        ('author name process', TfidfVectorizer(tokenizer=word_stemming, token_pattern=None, ngram_range=(1,2), lowercase=False), [1]),
        ('topic name process' , TfidfVectorizer(tokenizer=word_stemming, token_pattern=None, ngram_range=(1,2), lowercase=False), [3])
    ],
    remainder='passthrough', # do not touch the remaining data
    n_jobs=-1
)
rfc_transformer = ColumnTransformer(
    [
        ('drop process', 'drop', [0, 2, 10, 11, 12]),
        ('author name process', TfidfVectorizer(tokenizer=word_stemming, token_pattern=None, ngram_range=(1,2), lowercase=False), [1]),
        ('topic name process' , TfidfVectorizer(tokenizer=word_stemming, token_pattern=None, ngram_range=(1,2), lowercase=False), [3])
    ],
    remainder='passthrough', # do not touch the remaining data
    n_jobs=-1
)
voting_transformer = ColumnTransformer(
    [
        ('title name process', TfidfVectorizer(tokenizer=word_stemming, token_pattern=None, ngram_range=(1,2), lowercase=False), [0]),
        ('author name process', TfidfVectorizer(tokenizer=word_stemming, token_pattern=None, ngram_range=(1,2), lowercase=False), [1]),
        ('channel name process', TfidfVectorizer(tokenizer=word_stemming, token_pattern=None, ngram_range=(1,2), lowercase=False), [2]),
        ('topic name process' , TfidfVectorizer(tokenizer=word_stemming, token_pattern=None, ngram_range=(1,2), lowercase=False), [3])
    ],
    remainder='passthrough', # do not touch the remaining data
    n_jobs=-1
)

### 4. Model training
Feature 準備好了就可以訓練模型。本次 Competition 嘗試使用的 Model 有 XGBoost Classifier, LightGBM Classifier, CatBoost Classifier, AdaBoost Classifier, Random Forest Classifier 以及 VotingClassifier。VotingClassifier 會組合其他不同 model 的結果投票進行最終的預測，經過一連串的實驗，最後選擇將 XGBoost, LightGBM, CatBoost 搭配 Voting Classifier 作為最終的 model。

以下詳細說明各個 model 使用的 input features 及參數，還有最後的 validation 結果。每個 model 皆將 training set 分成 70% training set 以及 30% validation set 進行測試。

我們挑選參數的方式是透過GridSearchCV()，將我們想要測試的超參數，包含n_estimators, learning_rate, max_depth 等等，以list的方式去儲存，並且透過GridSearch所提供的API來對這些參數去做cross validation之計算，最後挑選一個在validation dataset上表現最好的一組參數，作為我們輸入參數的最佳解。

在這次的比賽中，為了節省訓練的時間，我們透過dump的方式，將模型及訓練好的權重給存起來，並且在需要inference的時候透過load的API，將模型讀進記憶體，直接去對我們分好的validation set去做inference.

* XGBoost (gamma=1.2, lambda=2.5, n_estimators=97, max_depth=7, learning_rate=0.141, random_state=0)  
    * features: author_name, topic, year, month, date, day, is_weekend, hour, num_video, len_content,sentiment_neg, sentiment_neu, sentiment_pos, sentiment_compound
    * [train score] 0.783027 (+/-0.003699)
    * [valid score] 0.589135 (+/-0.005340)

<!-- Add an empty line here -->

* LightGBM (n_estimators=231, objective='poisson', learning_rate=0.013, random_state=0)
    * features: author_name, topic, year, month, date, day, is_weekend, hour, num_video, len_content
    * [train score] 0.689440 (+/-0.000485)
    * [valid score] 0.593020 (+/-0.000761)

<!-- Add an empty line here -->

* CatBoost (n_estimators=700, depth=12, eval_metric='AUC', learning_rate=0.01, random_state=0)
    * features: author_name, topic, year, month, date, day, is_weekend, hour, num_video, len_content,sentiment_neg, sentiment_neu, sentiment_pos, sentiment_compound
    * [train score] 0.824449 (+/-0.006048)
    * [valid score] 0.594177 (+/-0.001094)

<!-- Add an empty line here -->

* AdaBoost (n_estimators=1500, estimator=DecisionTreeClassifier(max_depth=4), learning_rate=0.001, random_state=0)
    * features: author_name, topic, year, month, date, day, is_weekend, hour, num_video, len_content
    * [train score] 0.654205 (+/-0.008009)
    * [valid score] 0.586208 (+/-0.002057)
<!-- Add an empty line here -->  

* Random Forest (n_estimators=1150, max_depth=90, min_samples_leaf=1, random_state=0)
    * features: author_name, topic, year, month, date, day, is_weekend, hour, num_video, len_content,sentiment_neg, sentiment_neu, sentiment_pos, sentiment_compound
    * [train score] 0.998176 (+/-0.000360)
    * [valid score] 0.577714 (+/-0.000072)
<!-- Add an empty line here -->

* VotingClassifier (voting='soft', flatten_transform=True)
    * 使用 XGBoost、LightGBM做 voting 得到 training result
    * [train score] 0.887048 (+/-0.003507)
    * [valid score] 0.573097 (+/-0.001854)

In [7]:
# pipeline_xgb = Pipeline([('vect', xgb_transformer), ('clf', XGBClassifier())])
# pipeline_lgb = Pipeline([('vect', lgb_transformer), ('clf',  LGBMClassifier())])
# pipeline_cat = Pipeline([('vect', cat_transformer), ('clf',  CatBoostClassifier())])
# pipeline_ada = Pipeline([('vect', ada_transformer), ('clf',  AdaBoostClassifier())])
# pipeline_rfc = Pipeline([('vect', rfc_transformer), ('clf',  RandomForestClassifier())])
pipeline_xgb = XGBClassifier()
pipeline_lgb = LGBMClassifier()
pipeline_cat = CatBoostClassifier()
pipeline_ada = AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=0))
pipeline_rfc = RandomForestClassifier()

#### - To construct the training function

在我們建立的training function，我們透過cross_validate這個由sklearn所提供的API去計算roc_auc的validation score，並且最後會使用fit這個API來對model去做訓練。

In [8]:

def training(X_train, y_train, clf):

    score = cross_validate(clf, X_train, y_train, scoring='roc_auc', return_train_score=True, return_estimator=True, cv=2)
    print('train score: {:.6f} (+/-{:.6f})'.format(
        np.mean(score['train_score']), np.std(score['train_score'])))
    print('valid score: {:.6f} (+/-{:.6f})'.format(
        np.mean(score['test_score']), np.std(score['test_score'])))

    clf.fit(X_train, y_train)
    
    return clf, np.mean(score['train_score']), np.mean(score['test_score'])

#### - To contruct the grid search function

在這個grid search function我們除了對我們的參數去做gridsearch以尋找更好的參數外，我們也去將尋找到最好的結果存起來。

In [9]:
def grid_search_cv(ct, X_train, y_train, clf, param_grid, verbose_=False):
    
    X_train_ct = ct.fit_transform(X_train)
    
    # to report the grid search information
    if(verbose_):
        gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='roc_auc', n_jobs=-1, cv=2, return_train_score=True, verbose = 3)
    else:
        gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True)
    
    gs.fit(X_train_ct, y_train)
    
    results, idx = gs.cv_results_, gs.best_index_
    print('train score: {:.6f} (+/-{:.6f})'.format(results['mean_train_score'][idx], results['std_train_score'][idx]))
    print('valid score: {:.6f} (+/-{:.6f})'.format(results['mean_test_score'][idx], results['std_test_score'][idx]))
    print('best params:', gs.best_params_)
    return gs.best_params_, gs.best_estimator_, results['mean_test_score'][idx]

#### - to store the best parameter to a file

這個function去描述了如何去儲存一個模型的參數。

In [10]:
def parameter_storage(dict_path, file_name, best_param, best_validation, remaining_dict, best_estimator_list=None):
    if not os.path.exists(dict_path):
        os.makedirs(dict_path)
        
    file_path = os.path.join(dict_path, file_name + ".txt")
    
    with open(file_path, "w") as file:
        file.write(f'The features: {remaining_dict}\n')
        file.write(f'The best parameter: {best_param}\n')
        file.write(f'ngram_range : {ngram_range_}\n')
        if not (best_estimator_list==None):
            file.write('The best estimator_list: ')
            for i in range(len(best_estimator_list)):
                file.write(f' {best_estimator_list[i][0]}')
            file.write('\n')
        file.write(f'The best validation: {best_validation}\n')
        file.write('--------------------------------------------------------\n')

### 4-1. XGBOOST

In [ ]:
grid_search_enable = False

4-1-1. Grid sizing for XGBoost

In [11]:
param_grid_xgb = {
    'gamma' : [1.1, 1.2],
    'lambda' : [2.4, 2.5],
    'n_estimators': [97, 98],
    'max_depth': [7, 8],
    'learning_rate' : [0.14, 0.141]  
}

if (grid_search_enable):
    best_xgb_param, best_xgb, best_xgb_valid = grid_search_cv(xgb_transformer, df_train_x, df_train_y, pipeline_xgb, param_grid_xgb, True)
    

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 2/2] END gamma=1.1, lambda=2.4, learning_rate=0.14, max_depth=7, n_estimators=97;, score=(train=0.825, test=0.585) total time=   8.4s
[CV 1/2] END gamma=1.1, lambda=2.4, learning_rate=0.14, max_depth=7, n_estimators=97;, score=(train=0.821, test=0.584) total time=   8.9s
train score: 0.823161 (+/-0.002146)
valid score: 0.584373 (+/-0.000395)
best params: {'gamma': 1.1, 'lambda': 2.4, 'learning_rate': 0.14, 'max_depth': 7, 'n_estimators': 97}


4-1-2. Training for XGBoost

In [12]:
param_static_xgb = {
    'gamma' : 1.2,
    'lambda' : 2.5,
    'n_estimators': 97,
    'max_depth': 7,
    'learning_rate' : 0.141,
    'n_jobs' : -1,
    'random_state' : 0
}

if (grid_search_enable):
    xgboost = Pipeline([('vect', xgb_transformer), ('clf', best_xgb)])
else :
    xgboost = Pipeline([('vect', xgb_transformer), ('clf', XGBClassifier(**param_static_xgb))])
    
_ = training(df_train_x, df_train_y, xgboost)

train score: 0.827295 (+/-0.000225)
valid score: 0.585739 (+/-0.000656)


### 4-2. LightGBM

4-2-1. Grid sizing for LightGBM

In [13]:
param_grid_lgbm = {
    'learning_rate' : [0.13, 0.0131], 
    'n_estimators' : [230, 231],
    'objective' : ['regression', 'poisson']
}

if (grid_search_enable):
    best_lgbm_param, best_lgbm, best_lgbm_valid = grid_search_cv(lgb_transformer, df_train_x, df_train_y, pipeline_lgb, param_grid_lgbm, True)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV 1/2] END learning_rate=0.13, n_estimators=230, objective=regression;, score=(train=0.931, test=0.569) total time=   2.9s
[CV 2/2] END learning_rate=0.13, n_estimators=230, objective=regression;, score=(train=0.927, test=0.567) total time=   3.2s
[CV 1/2] END learning_rate=0.13, n_estimators=231, objective=regression;, score=(train=0.932, test=0.569) total time=   3.0s
[CV 1/2] END learning_rate=0.0131, n_estimators=230, objective=regression;, score=(train=0.732, test=0.592) total time=   3.1s
[CV 2/2] END learning_rate=0.13, n_estimators=230, objective=poisson;, score=(train=0.869, test=0.580) total time=   3.3s
[CV 2/2] END learning_rate=0.13, n_estimators=231, objective=regression;, score=(train=0.927, test=0.567) total time=   3.1s
[CV 1/2] END learning_rate=0.0131, n_estimators=230, objective=poisson;, score=(train=0.690, test=0.592) total time=   3.1s
[CV 1/2] END learning_rate=0.13, n_estimators=230, objective=poisso

4-2-2. Training for LightGBM

In [14]:
params_static_lgbm = {
    'random_state': 0, 
    'learning_rate' : 0.013,
    'n_estimators' : 230,
    'n_jobs' : -1,
    'objective' : 'poisson'
}

if (grid_search_enable):
    lgbm = Pipeline([('vect', lgb_transformer), ('clf', best_lgbm)])
else :
    lgbm = Pipeline([('vect', lgb_transformer), ('clf', LGBMClassifier(**params_static_lgbm))])


_ = training(df_train_x, df_train_y, lgbm)

train score: 0.689673 (+/-0.000349)
valid score: 0.591266 (+/-0.001339)


### 4-3. CatBoost

4-3-1. Grid sizing for Catboost

In [15]:
param_grid_cat = {
    'learning_rate' : [0.01], 
    'n_estimators' : [700],
    'depth' : [12]
}

if (grid_search_enable):
    best_cat_params, best_cat_estimator, best_cat_valid= grid_search_cv(cat_transformer, df_train_x, df_train_y, pipeline_cat, param_grid_cat, True)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
0:	learn: 0.6925111	total: 759ms	remaining: 8m 50s
0:	learn: 0.6926031	total: 817ms	remaining: 9m 31s
1:	learn: 0.6920114	total: 1.46s	remaining: 8m 28s
1:	learn: 0.6921981	total: 1.47s	remaining: 8m 33s
2:	learn: 0.6915026	total: 2.17s	remaining: 8m 25s
2:	learn: 0.6909394	total: 2.19s	remaining: 8m 29s
3:	learn: 0.6909922	total: 2.87s	remaining: 8m 20s
3:	learn: 0.6906376	total: 2.9s	remaining: 8m 25s
4:	learn: 0.6902948	total: 3.57s	remaining: 8m 16s
4:	learn: 0.6901053	total: 3.64s	remaining: 8m 25s
5:	learn: 0.6899069	total: 4.26s	remaining: 8m 12s
5:	learn: 0.6898346	total: 4.3s	remaining: 8m 17s
6:	learn: 0.6896002	total: 4.97s	remaining: 8m 12s
6:	learn: 0.6894964	total: 4.99s	remaining: 8m 13s
7:	learn: 0.6889079	total: 5.68s	remaining: 8m 11s
7:	learn: 0.6890035	total: 5.77s	remaining: 8m 19s
8:	learn: 0.6884298	total: 6.37s	remaining: 8m 9s
8:	learn: 0.6885649	total: 6.43s	remaining: 8m 13s
9:	learn: 0.6878762	total:

4-3-2. Training for CatBoost

In [16]:
params_static_cat = {
    'eval_metric' : 'AUC',
    'n_estimators' : 700,
    'depth' : 12,
    'learning_rate' : 0.01,
    'random_state' : 0,
    'verbose' : False
}

if (grid_search_enable):
    catboost = Pipeline([('ct', cat_transformer),('clf', best_cat_estimator)])
else :
    catboost = Pipeline([('ct', cat_transformer),('clf', CatBoostClassifier(**params_static_cat))])

_ = training(df_train_x, df_train_y, catboost)

0:	learn: 0.6923828	total: 400ms	remaining: 4m 39s
1:	learn: 0.6920220	total: 801ms	remaining: 4m 39s
2:	learn: 0.6914525	total: 1.21s	remaining: 4m 40s
3:	learn: 0.6909474	total: 1.63s	remaining: 4m 43s
4:	learn: 0.6906584	total: 2.04s	remaining: 4m 43s
5:	learn: 0.6903076	total: 2.45s	remaining: 4m 43s
6:	learn: 0.6899986	total: 2.86s	remaining: 4m 43s
7:	learn: 0.6896604	total: 3.27s	remaining: 4m 42s
8:	learn: 0.6890618	total: 3.69s	remaining: 4m 43s
9:	learn: 0.6886051	total: 4.08s	remaining: 4m 41s
10:	learn: 0.6883566	total: 4.5s	remaining: 4m 41s
11:	learn: 0.6878065	total: 4.91s	remaining: 4m 41s
12:	learn: 0.6875459	total: 5.31s	remaining: 4m 40s
13:	learn: 0.6869961	total: 5.71s	remaining: 4m 39s
14:	learn: 0.6865051	total: 6.13s	remaining: 4m 39s
15:	learn: 0.6860404	total: 6.54s	remaining: 4m 39s
16:	learn: 0.6855336	total: 6.95s	remaining: 4m 39s
17:	learn: 0.6852539	total: 7.38s	remaining: 4m 39s
18:	learn: 0.6845751	total: 7.8s	remaining: 4m 39s
19:	learn: 0.6842971	tot

### 4-4. AdaBoost

4-4-1. Grid sizing for AdaBoost

In [17]:
params_grid_ada = {
    'estimator__max_depth' : [4],
    'n_estimators': [1500],
    'learning_rate': [0.001],
    'random_state' : [0]
}

if (grid_search_enable):
    best_ada_params, best_ada_estimator, best_ada_valid = grid_search_cv(ada_transformer, df_train_x, df_train_y, pipeline_ada, params_grid_ada, True)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 2/2] END estimator__max_depth=4, learning_rate=0.001, n_estimators=1500, random_state=0;, score=(train=0.646, test=0.588) total time=16.2min
[CV 1/2] END estimator__max_depth=4, learning_rate=0.001, n_estimators=1500, random_state=0;, score=(train=0.662, test=0.584) total time=17.4min
train score: 0.654205 (+/-0.008009)
valid score: 0.586208 (+/-0.002057)
best params: {'estimator__max_depth': 4, 'learning_rate': 0.001, 'n_estimators': 1500, 'random_state': 0}


4-4-2. Training for AdaBoost

In [18]:
param_static_ada = {
    'estimator' : DecisionTreeClassifier(max_depth = 4), 
    'learning_rate' : 0.001, 
    'n_estimators' : 1560
}

if (grid_search_enable):
    adaBoost = Pipeline([('vect', ada_transformer), ('clf', best_ada_estimator)])
else :
    adaBoost = Pipeline([('vect', ada_transformer), ('clf', AdaBoostClassifier(**param_static_ada))])
    
_ = training(df_train_x, df_train_y, adaBoost)

train score: 0.655846 (+/-0.004560)
valid score: 0.581368 (+/-0.002565)


### 4-5. Random Forest Classifier

4-5-1. Grid sizing for RF

In [19]:

params_grid_rf = {
    'n_estimators' : [1150, 1200, 1250],
    'max_depth' : [90, 100, 110],
    'min_samples_leaf' : [1, 2]
}

if (grid_search_enable):
    best_RF_params, best_RF_estimator, best_RF_valid = grid_search_cv(rfc_transformer, df_train_x, df_train_y, pipeline_rfc, params_grid_rf, True)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END max_depth=90, min_samples_leaf=1, n_estimators=1150;, score=(train=0.998, test=0.577) total time= 6.1min
[CV 2/2] END max_depth=90, min_samples_leaf=1, n_estimators=1150;, score=(train=0.998, test=0.578) total time= 6.4min
train score: 0.998028 (+/-0.000017)
valid score: 0.577519 (+/-0.000554)
best params: {'max_depth': 90, 'min_samples_leaf': 1, 'n_estimators': 1150}


4-5-2. Training for RF

In [20]:
params_static_rf = {
    'n_jobs' : -1,
    'random_state' : 0,
    'n_estimators' : 1200,
    'max_depth' : 100,
    'min_samples_leaf': 1
}

if (grid_search_enable):
    RF = Pipeline([('vect', rfc_transformer), ('clf', best_RF_estimator)])
else :
    RF = Pipeline([('vect', rfc_transformer), ('clf', RandomForestClassifier(**params_static_rf))])

_ = training(df_train_x, df_train_y, RF)

train score: 0.999345 (+/-0.000056)
valid score: 0.573417 (+/-0.000696)


### 4-6. VotingClassifier

4-2-1. Grid sizing for Voting classifier

In [21]:
param_static_xgb = {
    'gamma' : 1.2,
    'lambda' : 2.5,
    'n_estimators': 97,
    'max_depth': 7,
    'learning_rate' : 0.141,
    'n_jobs' : -1,
    'random_state' : 0
}

params_static_LGBM = {
    'random_state': 0, 
    'learning_rate' : 0.013,
    'n_estimators' : 230,
    'n_jobs' : -1,
    'objective' : 'poisson'
}

params_static_cat = {
    'eval_metric' : 'AUC',
    'n_estimators' : 700,
    'depth' : 12,
    'learning_rate' : 0.01,
    'random_state' : 0,
    'verbose' : False
}

param_static_Adaboost = {
    'estimator' : DecisionTreeClassifier(max_depth = 4), 
    'learning_rate' : 0.001, 
    'n_estimators' : 1560
}

param_static_RF = {
    'n_jobs' : -1,
    'random_state' : 0,
    'n_estimators' : 1200,
    'max_depth' : 100,
    'min_samples_leaf': 1
}

voting_pipeline_xgb = Pipeline([('vect', xgb_transformer), ('clf', XGBClassifier(**param_static_xgb))])
voting_pipeline_lgb = Pipeline([('vect', lgb_transformer), ('clf',  LGBMClassifier(**params_static_LGBM))])
voting_pipeline_cat = Pipeline([('vect', cat_transformer), ('clf',  CatBoostClassifier(**params_static_cat))])
voting_pipeline_ada = Pipeline([('vect', ada_transformer), ('clf',  AdaBoostClassifier(**param_static_Adaboost))])
voting_pipeline_rfc = Pipeline([('vect', rfc_transformer), ('clf',  RandomForestClassifier(**param_static_RF))])

在這裡我們使用了2個classifier，分別是xgboost 和 lgbm

In [22]:
num_classifier = 2
# voting_estimator_list = [('pip1', pipeline_xgb), ('pip2', pipeline_lgb), ('pip3', pipeline_cat)]
voting_estimator_list = [('pip1', pipeline_xgb), ('pip2', pipeline_lgb)]

- to find the weight combinations

In [23]:
def weight_list_generator(num_classifier_):   
    weight_list = []

    # to control the weight
    binary_values = (1, 3, 5)
    weight_list = [list(i) for i in list(itertools.product(binary_values, repeat=num_classifier_))]

    print('weight list = ', weight_list)
    print('length of weight list = ', len(weight_list))
    return weight_list

In [24]:
weight_list = weight_list_generator(num_classifier_ = num_classifier)

weight list =  [[1, 1], [1, 3], [1, 5], [3, 1], [3, 3], [3, 5], [5, 1], [5, 3], [5, 5]]
length of weight list =  9


* multi-thread grid search for voting classifier

In [ ]:
param_grid_voting_static = {
    'estimators' : voting_estimator_list,
    'voting' : 'soft',
    'flatten_transform' : True, 
    'verbose' : True
}

lock = threading.Lock()


def process_weight(weight):
    # share in multiple threads
    global best_valid_score, best_weight, best_voting

    voting = Pipeline([('vect', voting_transformer), ('clf', VotingClassifier(**prarms_static_voting, weights=weight))])
    print('current weight =', weight)
    clf_voting, _, valid_voting = training(df_train_x, df_train_y, voting)

    # to protect the safety of shared variables
    with lock:
        if valid_voting > best_valid_score:
            best_valid_score = valid_voting
            best_weight = weight
            best_voting = clf_voting
    
    print(f'{weight} Finish!!')
    threadmax.release()
        
if (grid_search_enable):
    best_valid_score = 0
    best_weight = None
    best_voting = None
    mem = []
    threadmax = threading.BoundedSemaphore(32)
    
    for weight in weight_list:
        threadmax.acquire()
        thread = threading.Thread(target=process_weight, args=(weight,))
    
        thread.start()
        mem.append(thread)

    for thread in mem:
        thread.join()
        mem.remove(thread)
    
    print('end once')

    print('best_valid_score = %.6f' % best_valid_score)
    print('best_weight = ', best_weight)

In [26]:
# no n_jobs
prarms_static_voting = {
    'estimators' : voting_estimator_list, 
    'voting' : 'soft',
    'weights' : [1, 2],
    'flatten_transform' : True,
    'verbose' : True
}

if (grid_search_enable):
    voting = Pipeline([('vect', voting_transformer), ('clf', VotingClassifier(**param_grid_voting_static, weights=best_weight))])
    
else :
    voting = Pipeline([('vect', voting_transformer), ('clf', VotingClassifier(**prarms_static_voting))])

_ = training(df_train_x, df_train_y, voting)


[Voting] ..................... (1 of 2) Processing pip1, total=   2.1s
[Voting] ..................... (2 of 2) Processing pip2, total=   1.4s
[Voting] ..................... (1 of 2) Processing pip1, total=   2.0s
[Voting] ..................... (2 of 2) Processing pip2, total=   1.4s
train score: 0.887048 (+/-0.003507)
valid score: 0.573097 (+/-0.001854)
[Voting] ..................... (1 of 2) Processing pip1, total=   3.9s
[Voting] ..................... (2 of 2) Processing pip2, total=   2.6s


### 5. Testing data prediction

輸出預測結果，並且把結果存成CSV檔。

In [27]:
best_model = voting

y_score = best_model.predict_proba(df_test_x)[:, 1]

df_pred = pd.DataFrame({'Id': test_data['Id'], 'Popularity': y_score})
df_pred.to_csv('../output/test_pred.csv', index=False)

### 6.Results on Kaggle

經過多次嘗試不同的 Feature Extraction 與 Preprocessing，Model 的挑選以及參數的 tuning 後，在 Kaggle 平台上獲得了以下成績：

[Public Score] 0.5847   

[Private Score] 0.59706


### 7. Conclusion

這次的 Text Feature Engineering Competition 相當具有挑戰性，除了要在 News article 中以不同的技巧 parse 出可能有幫助的 features，還要針對這些feature去做extraction，（像是提取 HTML 中去除 tag 的結果、對單字純化的技巧、計算單字出現頻率以及重要性等等方式）。我們還有針對標題或是內文去做sentiment analysis的preprocessing。好在資訊越來越先進的時代，我們都有一些現成的函式庫（如：BeautifulSoup4，VanderSentiment等等），使用這些對新聞去做prediction是我們從來沒有想過的。

在多次嘗試各種 features 的可能性及組合，我們也利用了GridSearch來對超參數們做微調，以找到在我們dataset表現最好的參數們；利用param_storage還有儲存模型，這些調整參數的方法跟儲存模型權重的技巧也協助我們減少了大量的時間，讓我們有更多的機會去做feature extraction及嘗試各種的優化。

最後針對選擇 model 的部分則是使用了 XGBOOST, LightGBM, Catboost 得到了 performance 相當程度的提升，最後利用Voting來得到一個共同預測的結果，針對每個 model 所使用的 feature 和參數也不盡相同，最終透過多次實驗才得到了如此的成績，我們相信這些技巧未來在之後的Competition或是我們個人的研究上一定都會有很大的幫助。